In [1]:
import math
from timeit import default_timer as timer

from torch import Tensor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import TransformerDecoder, TransformerDecoderLayer
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from torch.nn.utils.rnn import pad_sequence
from typing import Iterable, List

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
def custom_loss(output, target, teacher_model=None, alpha=0.5, temperature=2.0):
    """
    Custom loss function for knowledge distillation.
    
    Parameters:
    output: Logits from the student model.
    target: Ground truth labels or logits from the teacher model.
    teacher_model: Pre-trained teacher model. If None, standard training is performed.
    alpha: Weighting factor for combining the teacher and student loss.
    temperature: Temperature for softening probabilities.
    """
    student_loss = F.cross_entropy(output, target.view(-1))
    
    if teacher_model:
        with torch.no_grad():
            teacher_output = teacher_model(target)
        teacher_output = teacher_output.view_as(output)

        # Soften the outputs of both student and teacher models
        soft_student_probs = F.log_softmax(output / temperature, dim=1)
        soft_teacher_probs = F.softmax(teacher_output / temperature, dim=1)

        # Calculate the distillation loss
        distillation_loss = F.kl_div(soft_student_probs, soft_teacher_probs, reduction='batchmean')

        # Combine the student and distillation loss
        return alpha * distillation_loss + (1 - alpha) * student_loss
    else:
        return student_loss


## Dataset and Data Processing

In [3]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

In [4]:
# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'de'

# Place-holders
token_transform = {}
vocab_transform = {}

In [6]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

In [7]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [8]:
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [9]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

In [10]:
# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [11]:
# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

In [12]:
# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

## Model
- We use a 6L6L Transformer as the base model.
- Embedding dimension is 1024. Each multi-head attention layer has 16 heads, with a dimension of 1024 for 
QKV if combining all the heads.
- The hidden projection dimension in FFNs is 4096.
- Dropout layers has a dropout rate of 0.1.
- We use a batch size of 1024.

In [13]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class SixLTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 1024,
                 dropout: float = 0.1):
        super(SixLTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [14]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 1024
NHEAD = 16
FFN_HID_DIM = 4096
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6

model = SixLTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(device)

## Optimizer
- Adam optimizer (Kingma & Ba, 2014) is used with β1 = 0.9 and β2 = 0.98.
- No weight decay is applied.
- Base learning rate is 0.001. 

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

## Scheduler
- We adopt a three-stage training scheme, where the learning rate (LR) of each stage decreases from base to zero following a cosine decay.
- The first LR cycle has 50000 steps, others have 88339 steps.
- A quantization event starts at the beginning of each stage.
- **We first train the model in float.**
- In the second stage, all weights will be binarized.
- In the last stage, both weights and activations will be binarized.

In [16]:
# Scheduler First Cycle
scheduler = CosineAnnealingLR(optimizer, T_max=50000)  # Adjust T_max based on training stages

# Scheduler Second and Third Cycle
# scheduler = CosineAnnealingLR(optimizer, T_max=88339)

## TODO: Loss with Knowledge Distillation
- We apply knowledge distillation (KD) during training.
- KD can be implemented by replacing the ground truth label in the cross-entropy loss function with the softmaxed logits from the teacher model, so it is optional for users.


In [17]:
# This is the normal Cross Entropy Loss, used by default when training the Transformer Architecture
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

## Training and Evaluation

In [18]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    
    # Update the learning rate
    scheduler.step()

    return losses / len(list(train_dataloader))

In [19]:
def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [20]:
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(model, optimizer)
    end_time = timer()
    val_loss = evaluate(model)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 5.637, Val loss: 5.068, Epoch time = 75.165s
Epoch: 2, Train loss: 4.427, Val loss: 4.279, Epoch time = 73.676s
Epoch: 3, Train loss: 3.847, Val loss: 3.974, Epoch time = 73.981s
Epoch: 4, Train loss: 3.528, Val loss: 3.869, Epoch time = 74.970s
Epoch: 5, Train loss: 3.325, Val loss: 3.724, Epoch time = 73.664s
Epoch: 6, Train loss: 3.146, Val loss: 3.636, Epoch time = 73.220s
Epoch: 7, Train loss: 3.007, Val loss: 3.614, Epoch time = 73.260s
Epoch: 8, Train loss: 2.884, Val loss: 3.531, Epoch time = 73.136s
Epoch: 9, Train loss: 2.753, Val loss: 3.466, Epoch time = 73.336s
Epoch: 10, Train loss: 2.632, Val loss: 3.459, Epoch time = 73.561s
Epoch: 11, Train loss: 2.537, Val loss: 3.468, Epoch time = 73.848s
Epoch: 12, Train loss: 2.419, Val loss: 3.392, Epoch time = 73.041s
Epoch: 13, Train loss: 2.304, Val loss: 3.328, Epoch time = 73.227s
Epoch: 14, Train loss: 2.204, Val loss: 3.312, Epoch time = 73.132s
Epoch: 15, Train loss: 2.105, Val loss: 3.320, Epoch time

## Testing it Works

In [24]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

In [25]:
# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [29]:
print(translate(model, "It is cold outside .")) 

 Ein Bauarbeiter wartet auf den nächsten Zug . 


Actual translation: `A construction worker waits for the next train`

**Translation is off, but at least words form coherent sentences. We would probably need to train a lot more for actual translations.**

## Binarization of Transformer Model
The function of casting floating-point values into binary Binarized Neural Machine Translation values is summarized as follows:

$$
\text{clip}(x, x_{\min}, x_{\max}) := \min(x_{\max}, \max(x_{\min}, x))
$$

$$
x_b := \left\lfloor \text{clip}\left( x, -1 + \varepsilon, 1 - \varepsilon \right) + 0.5 \right\rfloor \times B
$$

where $x$ is the input tensor, $\varepsilon$ is a small floating-point number that prevents overflow when taking the floor, and $B$ is the binarization bound.


In [35]:
def binarize_weights(model, epsilon=0.1):
    with torch.no_grad():
        for p in model.parameters():
            if p.dim() > 1:
                # Clip the weights to the range [-1 + ε, 1 - ε]
                clipped = p.data.clamp(-1 + epsilon, 1 - epsilon)
                # Scale and floor the weights to get binary values
                p.data = torch.floor((clipped + 1 - epsilon) * 0.5) * 2 - 1

# Initialize the model
binarized_model = SixLTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in binarized_model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# Binarize the weights of the model
binarize_weights(binarized_model)

binarized_model = binarized_model.to(device)


In [36]:
NUM_EPOCHS = 3

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(binarized_model, optimizer)
    end_time = timer()
    val_loss = evaluate(model)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 9.869, Val loss: 4.610, Epoch time = 72.122s
Epoch: 2, Train loss: 9.869, Val loss: 4.610, Epoch time = 72.486s
Epoch: 3, Train loss: 9.869, Val loss: 4.610, Epoch time = 72.014s
